In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.WEEKLY_CRYPTO_SPECULATION)

In [3]:
backtest_start_date = datetime(2021,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

## Historical Training and Prediction Set

In [5]:
market.connect()
training_sets = []
for ticker in tqdm(["BTC"]):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices,False)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
data = pd.concat(training_sets)
training_data = data.dropna().copy().sort_values(["year","week"])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.28it/s]


## Historical Modeling and Simulation Code

In [6]:
# historical model and simulation
pricer_class.db.connect()
model_start_date = backtest_start_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
model_end_date = backtest_end_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
for year in range(model_start_date.year,model_end_date.year):
    training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
    prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
    stuff = modeler_strat.model(training_slice,prediction_set,pricer_class.factors,False)
    stuff = stuff.rename(columns={"prediction":f"price_prediction"})
    relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
    stuff = pricer_class.sim_processor(stuff)
    pricer_class.db.store("sim",stuff[relevant_columns])
pricer_class.db.disconnect()

## Current Year Modeling Code

In [8]:
pricer_class.db.connect()
training_slice = training_data[(training_data["year"]<current_year) & (training_data["year"] >= current_year - 4)].reset_index(drop=True)
prediction_set = training_data[training_data["year"]==current_year].reset_index(drop=True)
stuff = modeler_strat.recommend_model(training_slice,pricer_class.factors,False)
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]
stuff["year"] = current_year
pricer_class.db.store("models",stuff)
pricer_class.db.disconnect()